In [1]:
import pandas as pd
import numpy as np
import datetime
import os
from fuzzywuzzy import fuzz
print(os.getcwd())
print(datetime.datetime.now())

/home/jian/Projects/Big_Lots/Analysis/2020_Q1/plannder_requests/BR_adding_DMA_20200309
2020-03-09 14:15:48.809944


/usr/local/lib/python3.6/site-packages/fuzzywuzzy/fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
df_DMA_zip=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",
                         skiprows=1,dtype=str,usecols=[0,2,6,9])
print(df_DMA_zip.shape)
df_DMA_zip.columns=['Zip','DMA',"State",'County']
df_DMA_zip=df_DMA_zip.drop_duplicates()
df_DMA_zip['County']=df_DMA_zip['County'].str.lower()
print(df_DMA_zip.shape)

df_DMA_list_by_zip=df_DMA_zip.groupby("Zip")['DMA'].apply(set).to_frame().reset_index()
df_DMA_list_by_zip=df_DMA_list_by_zip.rename(columns={"DMA":"DMA_list"})
df_DMA_zip.head(2)

(51324, 4)
(51324, 4)


,Zip,DMA,State,County
0,00501,NEW YORK,NY,suffolk
1,00544,NEW YORK,NY,suffolk


In [3]:
df_BR_file=pd.read_excel("./BL Big Lots stores in Art Van Zips 2020 0306.xlsx",
                         sheet_name="Art Van",dtype=str)
df_BR_file=df_BR_file[['location_id','C_county','C_city','Zip','ST']].drop_duplicates()
df_BR_file['C_county']=df_BR_file['C_county'].apply(lambda x: x.replace(" County","").lower())
print(df_BR_file.shape)
print(df_BR_file['location_id'].nunique())
df_BR_file.head(2)

(183, 5)
84


,location_id,C_county,C_city,Zip,ST
0,30,hancock,Findlay,43402,OH
1,45,ottawa,Genoa,43616,OH


In [4]:
store_list=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20200301-134228-900.txt",
                        dtype=str,sep="|")
store_list[store_list['location_id']=="4619"]

,location_id,location_desc,open_dt,close_dt,address_line_1,address_line_2,city_nm,state_nm,zip_cd,longitude_meas,latitude_meas,sales_square_feet,furniture_sales_square_feet


In [5]:
df_4619=df_BR_file[pd.isnull(df_BR_file['Zip'])]
df_BR_file=df_BR_file[pd.notnull(df_BR_file['Zip'])]
df_BR_file['Zip']=df_BR_file['Zip'].apply(lambda x: x.split("-")[0].zfill(5))
df_4619

,location_id,C_county,C_city,Zip,ST
233,4619,kane,Algonquin,NaN,NaN


In [6]:
df_DMA_zip[(df_DMA_zip['County'].str.lower()=="kane") & (df_DMA_zip['State']=="IL")]['DMA'].unique()

array(['CHICAGO'], dtype=object)

In [7]:
df_4619['DMA']="CHICAGO"
df_4619['DMA_list']=set(["CHICAGO"])
df_4619

,location_id,C_county,C_city,Zip,ST,DMA,DMA_list
233,4619,kane,Algonquin,NaN,NaN,CHICAGO,CHICAGO


In [8]:
df_BR_file=pd.merge(df_BR_file,df_DMA_zip,on="Zip",how="left")
df_BR_file.head(2)

,location_id,C_county,C_city,Zip,ST,DMA,State,County
0,30,hancock,Findlay,43402,OH,TOLEDO,OH,wood
1,45,ottawa,Genoa,43616,OH,TOLEDO,OH,lucas


In [9]:
df_BR_file['match_1']=df_BR_file['C_county']+", "+df_BR_file['ST']
df_BR_file['match_2']=df_BR_file['County']+", "+df_BR_file['State']
df_BR_file['score']=df_BR_file.apply(lambda x: fuzz.partial_ratio(x['match_1'],x['match_2']),axis=1)
df_BR_file=df_BR_file.sort_values(["location_id","score"],ascending=[True,False])
df_BR_file=df_BR_file.drop_duplicates(["location_id","score"])
print(df_BR_file.shape)
del df_BR_file['ST']
del df_BR_file['County']
del df_BR_file['match_1']
del df_BR_file['match_2']
del df_BR_file['score']
print(df_BR_file.shape)
df_BR_file.head(2)

(121, 11)
(121, 6)


,location_id,C_county,C_city,Zip,DMA,State
51,1006,kane,Batavia,29577,MYRTLE BEACH-FLORENCE,SC
52,1006,dupage,Glendale Heights,29577,MYRTLE BEACH-FLORENCE,SC


In [10]:
df_BR_file=pd.merge(df_BR_file,df_DMA_list_by_zip,on="Zip",how="left")
df_BR_file.head(2)

,location_id,C_county,C_city,Zip,DMA,State,DMA_list
0,1006,kane,Batavia,29577,MYRTLE BEACH-FLORENCE,SC,{MYRTLE BEACH-FLORENCE}
1,1006,dupage,Glendale Heights,29577,MYRTLE BEACH-FLORENCE,SC,{MYRTLE BEACH-FLORENCE}


In [11]:
df_4619

,location_id,C_county,C_city,Zip,ST,DMA,DMA_list
233,4619,kane,Algonquin,NaN,NaN,CHICAGO,CHICAGO


In [12]:
df_output=df_BR_file.append(df_4619)
df_output['location_id']=df_output['location_id'].astype(int)
df_output=df_output.sort_values("location_id")

/home/jian/.local/lib/python3.6/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [13]:
df_output['location_id'].nunique()

84

In [14]:
df_output.to_csv("./BL_DMA_for_121_JL_"+str(datetime.datetime.now())+".csv",index=False)

In [15]:
# QC_49307
df_DMA_zip[df_DMA_zip['Zip']=="49307"]

,Zip,DMA,State,County
26165,49307,GRAND RAPIDS-KALMZOO-B.CRK,MI,newaygo
26166,49307,TRAVERSE CITY-CADILLAC,MI,mecosta
